In [5]:
import os
from os.path import isfile
import sys
import glob

os.environ["CUDA_VISIBLE_DEVICES"]="2"
#os.environ["TF_FORCE_GPU_ALLOW_GROWTH"]="true"
os.environ["TF_GPU_ALLOCATOR"]="cuda_malloc_async"

import numpy as np
import matplotlib.pyplot as plt

import tensorflow as tf
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.layers import Input, Dense, Concatenate, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.regularizers import L1, L2
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from sklearn.preprocessing import MinMaxScaler

import mlflow
import mlflow.tensorflow

2024-09-05 08:23:58.770222: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-09-05 08:23:58.876659: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.


In [6]:
mlflow.tensorflow.autolog()
mlflow.set_experiment("ATMS fixed q Reduced all levels new")

2024/09/05 08:24:08 INFO mlflow.tracking.fluent: Experiment with name 'ATMS fixed q Reduced all levels new' does not exist. Creating a new experiment.


<Experiment: artifact_location='file:///data/nature_run/work/src/mlruns/816335408861974466', creation_time=1725539048234, experiment_id='816335408861974466', last_update_time=1725539048234, lifecycle_stage='active', name='ATMS fixed q Reduced all levels new', tags={}>

In [ ]:
#train = ['/data/nature_run/fulldays_reduced/all_20060803.npz', '/data/nature_run/fulldays_reduced/all_20060815.npz', '/data/nature_run/fulldays_reduced/all_20060915.npz']
#test = ['/data/nature_run/fulldays_reduced/all_20060315.npz', '/data/nature_run/fulldays_reduced/all_20060515.npz', '/data/nature_run/fulldays_reduced/all_20060615.npz', '/data/nature_run/fulldays_reduced/all_20060715.npz', '/data/nature_run/fulldays_reduced/all_20061015.npz', '/data/nature_run/fulldays_reduced/all_20061115.npz', '/data/nature_run/fulldays_reduced/all_20061215.npz']

#print(len(train), len(test), len(train)+ len(test)) 


In [ ]:
files = glob.glob("/data/nature_run/fulldays_reduced_evenmore/*.npz")
print(files)

In [ ]:
train = ["/data/nature_run/fulldays_reduced_evenmore/20060615/hsel.npy",
         "/data/nature_run/fulldays_reduced_evenmore/20061215/hsel.npy",
         "/data/nature_run/fulldays_reduced_evenmore/20060515/hsel.npy",
         "/data/nature_run/fulldays_reduced_evenmore/20060815/hsel.npy",
         "/data/nature_run/fulldays_reduced_evenmore/20060915/hsel.npy",
         "/data/nature_run/fulldays_reduced_evenmore/20060715/hsel.npy",
         "/data/nature_run/fulldays_reduced_evenmore/20061015/hsel.npy",
         "/data/nature_run/fulldays_reduced_evenmore/20060315/hsel.npy",
         "/data/nature_run/fulldays_reduced_evenmore/20061115/hsel.npy"]

test = ["/data/nature_run/fulldays_reduced_evenmore/20060803/hsel.npy"]


In [9]:
#AE

def dataload(instr):

    train = ["/data/nature_run/fulldays_reduced_evenmore/20060615/mh.npy",
             "/data/nature_run/fulldays_reduced_evenmore/20061215/mh.npy",
             "/data/nature_run/fulldays_reduced_evenmore/20060515/mh.npy",
             "/data/nature_run/fulldays_reduced_evenmore/20060815/mh.npy",
             "/data/nature_run/fulldays_reduced_evenmore/20060915/mh.npy",
             "/data/nature_run/fulldays_reduced_evenmore/20060715/mh.npy",
             "/data/nature_run/fulldays_reduced_evenmore/20061015/mh.npy",
             "/data/nature_run/fulldays_reduced_evenmore/20060315/mh.npy",
             "/data/nature_run/fulldays_reduced_evenmore/20061115/mh.npy"]

    test = ["/data/nature_run/fulldays_reduced_evenmore/20060803/mh.npy"]

    if instr == "hsel":
        train = [x.replace("mh", instr) for x in train]
        test = [x.replace("mh", instr) for x in test]

    dat_train = []
    scalar_train = []
    table_train = []
    
    dat_test = []
    scalar_test = []
    table_test = []
    
    for i in train:
        # Keys: 'mh', 'hsel', 'scalar', 'table']
        tmp = np.load(i)
        dat_train.append(tmp)
        
        tmp = np.load(i.replace(instr, "table"))
        table_train.append(tmp)
    
        tmp = np.load(i.replace(instr, "scalar"))
        scalar_train.append(tmp)
    
    
    dat_train = np.vstack(dat_train)
    table_train = np.vstack(table_train)
    scalar_train = np.vstack(scalar_train)
        
    for i in test:
        # Keys: 'mh', 'hsel', 'scalar', 'table']
        tmp = np.load(i)
        dat_test.append(tmp)
        
        tmp = np.load(i.replace(instr, "table"))
        table_test.append(tmp)
    
        tmp = np.load(i.replace(instr, "scalar"))
        scalar_test.append(tmp)
    
    
    dat_test = np.vstack(dat_test)
    table_test = np.vstack(table_test)
    scalar_test = np.vstack(scalar_test)
    
    return dat_train, table_train, scalar_train, dat_test, table_test, scalar_test


In [ ]:
def get_data(filelist):
    hsel_val = []
    scalar_val = []
    table_val = []

    for i in filelist:
        # Keys: 'mh', 'hsel', 'scalar', 'table']
        tmp = np.load(i)
        hsel_val.append(tmp["mh"])
        scalar_val.append(tmp["scalar"])
        table_val.append(tmp["table"])

    return np.vstack(hsel_val), np.vstack(scalar_val), np.vstack(table_val)


train = ['/data/nature_run/fulldays_reduced_evenmore/all_20060815.npz',
         '/data/nature_run/fulldays_reduced_evenmore/all_20060915.npz',
         '/data/nature_run/fulldays_reduced_evenmore/all_20061015.npz',
         '/data/nature_run/fulldays_reduced_evenmore/all_20060515.npz',
         '/data/nature_run/fulldays_reduced_evenmore/all_20060715.npz',
         '/data/nature_run/fulldays_reduced_evenmore/all_20061115.npz',
         '/data/nature_run/fulldays_reduced_evenmore/all_20060315.npz', 
         '/data/nature_run/fulldays_reduced_evenmore/all_20061215.npz',
         '/data/nature_run/fulldays_reduced_evenmore/all_20060615.npz']

test = ['/data/nature_run/fulldays_reduced_evenmore/all_20060803.npz']

validation = ['/data/nature_run/fulldays_reduced_evenmore/all_20060803.npz']

hsel_train, scalar_train, table_train = get_data(train)
hsel_test, scalar_test, table_test = get_data(test)
hsel_val, scalar_val, table_val = get_data(validation)

In [17]:
hsel_train, table_train, scalar_train, hsel_test, table_test, scalar_test = dataload("mh")

In [18]:
hsel_train, table_train, scalar_train, hsel_test, table_test, scalar_test = dataload("mh")
hsel_val = hsel_test.copy()
table_val = table_test.copy()
scalar_val = scalar_test.copy()


In [19]:
print(hsel_train.shape)
print(table_train.shape)

(8742704, 22)
(8742704, 72, 3)


In [ ]:
# spress_test = scalar_test[:, 3].reshape(-1, 1)
# spress_train = scalar_train[:, 3].reshape(-1, 1)
# spress_val = scalar_val[:, 3].reshape(-1, 1)


# spress_test
# spress_train
# spress_val

# q_train = table_train[:, :, 1]
# q_test = table_test[:, :, 1]
# q_val = table_val[:, :, 1]

# mins = np.min(hsel_val, axis=0)
# maxs = np.max(hsel_val, axis=0)
# hsel_train = (hsel_val - mins)/(maxs - mins)
# np.savez("hsel_val_scalar.npz", mins=mins, maxs=maxs)

# mins = np.min(hsel_train, axis=0)
# maxs = np.max(hsel_train, axis=0)
# hsel_train = (hsel_train - mins)/(maxs - mins)
# np.savez("hsel_train_scalar.npz", mins=mins, maxs=maxs)

# mins = np.min(hsel_test, axis=0)
# maxs = np.max(hsel_test, axis=0)
# hsel_test = (hsel_test - mins)/(maxs - mins)
# np.savez("hsel_test_scalar.npz", mins=mins, maxs=maxs)

In [20]:
# What should we do
# 1) Only use train scalars
# 2) Scale seperateley


# Presssure
spress_test = scalar_test[:, 3].reshape(-1, 1)
spress_train = scalar_train[:, 3].reshape(-1, 1)
spress_val = scalar_val[:, 3].reshape(-1, 1)

mins = np.min(spress_train, axis=0)
maxs = np.max(spress_train, axis=0)
np.savez("spress_scalar.npz", mins=mins, maxs=maxs)
spress_test = (spress_test - mins)/(maxs - mins)
spress_train = (spress_train - mins)/(maxs - mins)
spress_val = (spress_val - mins)/(maxs - mins)


# Labels
q_train = table_train[:, :, 2]
q_test = table_test[:, :, 2]
q_val = table_val[:, :, 2]

# Spectra

mins = np.min(hsel_train, axis=0)
maxs = np.max(hsel_train, axis=0)

np.savez("minimac_scaling_factors_mh.npz", maxs=maxs, mins=mins)
#s_factors = np.load("minimac_scaling_factors_hsel.npz")
#mins = s_factors['mins']
#maxs = s_factors['maxs']


hsel_train = np.nan_to_num(hsel_train)
hsel_test = np.nan_to_num(hsel_test)
hsel_val = np.nan_to_num(hsel_val)

hsel_train = (hsel_train - mins)/(maxs - mins)
hsel_test = (hsel_test - mins)/(maxs - mins)
hsel_val = (hsel_val - mins)/(maxs - mins)


with tf.device('/cpu:0'):
    # train
    x_train = {'rad': tf.convert_to_tensor(hsel_train, np.float32),
               'spress': tf.convert_to_tensor(spress_train, np.float32)}
    del hsel_train
    del spress_train
    
    y_train =  tf.convert_to_tensor(q_train, np.float32)
    del q_train

    
    # Val
    x_val = {'rad': tf.convert_to_tensor(hsel_val, np.float32),
             'spress': tf.convert_to_tensor(spress_val, np.float32)}
    del hsel_val
    del spress_val
    
    y_val =  tf.convert_to_tensor(q_val, np.float32)
    del q_val

    
    # Test
    x_test = {'rad': tf.convert_to_tensor(hsel_test, np.float32),
              'spress': tf.convert_to_tensor(spress_test, np.float32)}
    del hsel_test
    del spress_test
    
    y_test =  tf.convert_to_tensor(q_test, np.float32)
    del q_test


2024-09-05 08:29:12.087517: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-09-05 08:29:12.633490: I tensorflow/core/common_runtime/gpu/gpu_process_state.cc:222] Using CUDA malloc Async allocator for GPU: 0
2024-09-05 08:29:12.634479: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1616] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 31017 MB memory:  -> device: 0, name: Tesla V100-SXM2-32GB, pci bus id: 0000:3d:00.0, compute capability: 7.0


In [ ]:
# print(x_test)
# print(x_val)
# print(y_train.shape)
#a = np.argwhere(np.isnan(x_train['rad']))
#print(np.sum(np.isnan(x_train['rad'])))
#np.nan_to_num(x_train['rad'], copy=False)
#print(np.sum(np.isnan(x_train['rad'])))
#print(a)
#print(len(a))

np.sum(np.isnan(x_train['rad']))

In [21]:
def build_ae_model(config):
    # ATMS 22
    mh = Input(shape=(config["shape"],), name="rad")
    spress = Input(shape=(1,), name="spress")
    # [ha, hb, hc, hd, hw, mh]

    encoder = load_model("encoder_3_mae.keras")
    encoder.trainable = False
    #for layer in encoder.layers:
    #    layer.trainable = False

    
    enc = encoder(mh)
    x = Concatenate()([enc, spress])
    for i in range(config["num_layers"]):
        x =  Dropout(config["dropout"])(Dense(config["num_neurons"], 
                                             activation=config["activation"])(x))
    outputs = Dense(config['output'], name="Temp")(x)

    model = Model(inputs=[mh, spress], outputs=outputs, name="Temp")
    model.compile(optimizer="adam", loss='mae')

    return model

config = {'shape': 1957,
          'output': 72,
          'dropout': np.random.choice([0.01, 0.1, 0.2, 0.5]),
          'num_layers': np.random.choice([3, 4, 5, 10]),
          'num_neurons': np.random.choice([16, 32, 64, 128]),
          'activation': np.random.choice(['gelu', 'gelu', 'relu'])}

model = build_ae_model(config)
model.summary()


Model: "Temp"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 rad (InputLayer)               [(None, 1957)]       0           []                               
                                                                                                  
 model_1 (Functional)           (None, 64)           275468      ['rad[0][0]']                    
                                                                                                  
 spress (InputLayer)            [(None, 1)]          0           []                               
                                                                                                  
 concatenate (Concatenate)      (None, 65)           0           ['model_1[0][0]',                
                                                                  'spress[0][0]']              

In [22]:
def build_model(config):
    # ATMS 22
    mh = Input(shape=(config["shape"],), name="rad")
    spress = Input(shape=(1,), name="spress")
    # [ha, hb, hc, hd, hw, mh]
    
    x = Concatenate()([mh, spress])
    for i in range(config["num_layers"]):
        x =  Dropout(config["dropout"])(Dense(config["num_neurons"], 
                                             activation=config["activation"])(x))
    outputs = Dense(config['output'], name="Temp")(x)

    model = Model(inputs=[mh, spress], outputs=outputs, name="Temp")
    model.compile(optimizer="adam", loss='mae')

    return model

config = {'shape': 22,
          'output': 72,
          'dropout': np.random.choice([0.01, 0.1, 0.2, 0.5]),
          'num_layers': np.random.choice([3, 4, 5, 10]),
          'num_neurons': np.random.choice([16, 32, 64, 128]),
          'activation': np.random.choice(['gelu', 'gelu', 'relu'])}

model = build_model(config)
model.summary()


Model: "Temp"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 rad (InputLayer)               [(None, 22)]         0           []                               
                                                                                                  
 spress (InputLayer)            [(None, 1)]          0           []                               
                                                                                                  
 concatenate_1 (Concatenate)    (None, 23)           0           ['rad[0][0]',                    
                                                                  'spress[0][0]']                 
                                                                                                  
 dense_3 (Dense)                (None, 32)           768         ['concatenate_1[0][0]']       

In [23]:
print("DING")

DING


In [ ]:
# AE

for _ in range(100):
    config = {'shape': 22,
              'output': 72,
              'dropout': np.random.choice([0.01, 0.1, 0.2, 0.5]),
              'num_layers': np.random.choice([3, 4, 5, 10]),
              'num_neurons': np.random.choice([16, 32, 64, 128]),
              'activation': np.random.choice(['gelu', 'gelu', 'relu'])}
    
    ae = False
    #ae = np.random.choice([True, False, True, True])
    #if ae:
    #    model = build_ae_model(config)
    #else:
    model = build_model(config)
    
    with mlflow.start_run() as run:
        model_checkpoint_callback = ModelCheckpoint(
            filepath=f"models/model_{run.info.run_id}.keras",
            save_weights_only=False,
            monitor='val_loss',
            mode='min',
            save_best_only=True)
        callback = EarlyStopping(monitor='val_loss', patience=8, verbose=0)
        mlflow.log_param("Autoencoder", ae)

        history = model.fit(x_train, y_train, epochs=1000, batch_size=2000,
                            validation_data=(x_val, y_val),
                            verbose=0,
                            callbacks=[callback, model_checkpoint_callback])
        
        evaled_test = model.evaluate(x_train, y_train, batch_size=100, verbose=0)
        evaled_train = model.evaluate(x_test, y_test, batch_size=100, verbose=0)

        mlflow.log_metric("test_loss", evaled_test)
        mlflow.log_metric("train_loss", evaled_train)

        plt.plot(history.history["loss"])
        plt.plot(history.history["val_loss"])
        plt.title('Model Loss')
        plt.ylabel('Loss')
        plt.xlabel('Epoch')
        plotname = f"plots/loss_plot_{run.info.run_id}.png"
        plt.savefig(plotname)
        plt.close()

        # Log the plot as an artifact
        mlflow.log_artifact(plotname)
        #model_path = "modles"
        #mlflow.tensorflow.log_model(tf_saved_model_dir=model_path, artifact_path="model")


2024/09/05 08:32:04 WARNING mlflow.tensorflow: Unrecognized dataset type <class 'dict'>. Dataset logging skipped.
2024/09/05 08:32:04 WARNING mlflow.tensorflow: Failed to log training dataset information to MLflow Tracking. Reason: 'tensorflow.python.framework.ops.EagerTensor' object has no attribute 'flatten'


1/1 [==============================] - 0s 101ms/step
INFO:tensorflow:Assets written to: /tmp/tmpk_osr46h/model/data/model/assets


2024/09/05 08:36:04 WARNING mlflow.utils.autologging_utils: MLflow autologging encountered a warning: "/data/jmackin1/miniconda/envs/tf-gpu/lib/python3.10/site-packages/_distutils_hack/__init__.py:33: UserWarning: Setuptools is replacing distutils."
2024/09/05 08:39:10 WARNING mlflow.tensorflow: Unrecognized dataset type <class 'dict'>. Dataset logging skipped.
2024/09/05 08:39:10 WARNING mlflow.tensorflow: Failed to log training dataset information to MLflow Tracking. Reason: 'tensorflow.python.framework.ops.EagerTensor' object has no attribute 'flatten'


1/1 [==============================] - 0s 78ms/step
INFO:tensorflow:Assets written to: /tmp/tmph238xi7z/model/data/model/assets


2024/09/05 08:45:49 WARNING mlflow.tensorflow: Unrecognized dataset type <class 'dict'>. Dataset logging skipped.
2024/09/05 08:45:49 WARNING mlflow.tensorflow: Failed to log training dataset information to MLflow Tracking. Reason: 'tensorflow.python.framework.ops.EagerTensor' object has no attribute 'flatten'


...

2024/09/05 12:52:59 WARNING mlflow.utils.environment: Encountered an unexpected error while inferring pip requirements (model URI: /tmp/tmpkw2rfnx9/model, flavor: tensorflow). Fall back to return ['tensorflow==2.10.0']. Set logging level to DEBUG to see the full traceback. 
2024/09/05 12:56:16 WARNING mlflow.tensorflow: Unrecognized dataset type <class 'dict'>. Dataset logging skipped.
2024/09/05 12:56:16 WARNING mlflow.tensorflow: Failed to log training dataset information to MLflow Tracking. Reason: 'tensorflow.python.framework.ops.EagerTensor' object has no attribute 'flatten'


1/1 [==============================] - 0s 86ms/step
INFO:tensorflow:Assets written to: /tmp/tmp8xfmhe7e/model/data/model/assets


2024/09/05 13:03:36 WARNING mlflow.utils.environment: Encountered an unexpected error while inferring pip requirements (model URI: /tmp/tmp8xfmhe7e/model, flavor: tensorflow). Fall back to return ['tensorflow==2.10.0']. Set logging level to DEBUG to see the full traceback. 
2024/09/05 13:06:37 WARNING mlflow.tensorflow: Unrecognized dataset type <class 'dict'>. Dataset logging skipped.
2024/09/05 13:06:37 WARNING mlflow.tensorflow: Failed to log training dataset information to MLflow Tracking. Reason: 'tensorflow.python.framework.ops.EagerTensor' object has no attribute 'flatten'


1/1 [==============================] - 0s 116ms/step
INFO:tensorflow:Assets written to: /tmp/tmpsdcgmmkq/model/data/model/assets


2024/09/05 13:19:38 WARNING mlflow.utils.environment: Encountered an unexpected error while inferring pip requirements (model URI: /tmp/tmpsdcgmmkq/model, flavor: tensorflow). Fall back to return ['tensorflow==2.10.0']. Set logging level to DEBUG to see the full traceback. 
2024/09/05 13:22:58 WARNING mlflow.tensorflow: Unrecognized dataset type <class 'dict'>. Dataset logging skipped.
2024/09/05 13:22:58 WARNING mlflow.tensorflow: Failed to log training dataset information to MLflow Tracking. Reason: 'tensorflow.python.framework.ops.EagerTensor' object has no attribute 'flatten'
